# Brewery Tutorial
__[Open Brewery](https://www.openbrewerydb.org)__ DB is a free dataset and API with public information on breweries, cideries, brewpubs, and bottleshops. The goal of Open Brewery DB is to maintain an open-source, community-driven dataset and provide a public API. Datasets provided by the project are available in the following formats:
- __[CSV](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.csv)__
- __[JSON](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.json)__
- __[PostgreSQL SQL](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.sql)__

For this tutorial, CSV will be used.

TerminusDB Server must be installed on your system before running the Python script. Follow the instructions on __[terminusdb-bootstrap](https://github.com/terminusdb/terminusdb-bootstrap)__. terminusdb-server will be running as a Docker container on http://127.0.0.1:6363.

Python client of TerminusDB is also required. It can be installed from source or through `pip`, you can follow the instructions in the __[repository](https://github.com/terminusdb/terminusdb-client-python)__. When running `pip install terminusdb-client[dataframe]` you will get pandas that is required for reading and importing data from CSV files. If installed from source, run `pip install pandas`. tdqm is used for adding a progress bar, run `pip install tqdm` to install it.

## Import libraries
For transfering data from a CSV file to a TerminusDB database, the Python client of TerminusDB and pandas are required. A progress bar is added to the script using tqdm. To import these libraries, add the following lines:

In [ ]:
from typing import List, Optional
from terminusdb_client import WOQLQuery, WOQLClient
from terminusdb_client.woqlschema.woql_schema import (
    DocumentTemplate,
    EnumTemplate,
    WOQLSchema,
)

import pandas as pd
from tqdm import tqdm

## Database management
TerminusDB Server must be installed on your system before running the Python script. Follow the instructions on __[terminusdb-bootstrap](https://github.com/terminusdb/terminusdb-bootstrap)__. terminusdb-server will be running as a Docker container on http://127.0.0.1:6363.

Using the Python client:
- Establish a connection to TerminusDB
- Create a database named *open_brewery*
- Insert schema into database
- Import data from CSV file
- Print headers and stats
- Print data from TerminusDB

In [ ]:
if __name__ == "__main__":
    db_id = "Brewery"
    url = "https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv"
    client = WOQLClient("http://127.0.0.1:6363")
    client.connect()
    try:
        client.create_database(db_id, team="admin", label = "Open Brewery Graph", description = "Create a graph with brewery data")
    except Exception:
        client.set_db(db_id)
    client.insert_document(my_schema.to_dict(),
                           graph_type="schema",
                           commit_msg="I am checking in the schema")
    csv_info(url)
    insert_data(client, url)
    results = client.get_all_documents(graph_type="instance", count=10)
    print("\nRESULTS\n", list(results))

## Schema creation
The dataset has the following columns:
- obdb_id
- name
- brewery_type
- street
- address_2
- address_3
- city
- state
- county_province
- postal_code
- website_url
- phone
- created_at
- updated_at
- country
- longitude
- latitude
- tags

Some of which are optional and rarely have a value assigned and can be omitted when creating the schema and importing the values.

Analyzing the dataset:

- A brewery has *name*, *type*, *address*, *phone* and *website url*
- A brewery can be any of eleven different types (micro, nano, regional, brewpub, large, planning, bar, contract, proprietor, closed, taproom)
- An address is a group of values that include *street*, *city*, *state*, *country*, *postal code* and *coordinates*
- Coordinates are a pair of values, longitude and latitude

Based on what's described above, the following documents are created, each class represents a document in the schema except Brewery Type that is an enum:
- Brewery
- Brewrey_Type
- Address
- City
- State
- Country
- Coordinates

`my_schema` is a `WOQLSchema` object that contains the schema itself.

In [ ]:
my_schema = WOQLSchema()

class Address(DocumentTemplate):
    """This is address"""

    _subdocument = []
    _schema = my_schema
    city: "City"
    state: Optional["State"]
    country: "Country"
    coordinates: List["Coordinates"]
    postal_code: str
    street: str

class Brewery(DocumentTemplate):
    _schema = my_schema
    address_of: "Address"
    name: str
    phone: str
    type_of: "Brewery_Type"
    website_url: str

class Brewery_Type(EnumTemplate):
    _schema = my_schema
    micro = ()
    nano = ()
    regional = ()
    brewpub = ()
    large = ()
    planning = ()
    bar = ()
    contract = ()
    proprietor = ()
    closed = ()
    taproom = ()

class City(DocumentTemplate):
    _schema = my_schema
    name: str

class Coordinates(DocumentTemplate):
    _schema = my_schema
    latitude: float
    longitude: float

class Country(DocumentTemplate):
    _schema = my_schema
    name: str

class State(DocumentTemplate):
    _schema = my_schema
    name: str

## Print CSV info
Before transfering data to TerminusDB, headers and stats of the dataset are printed by calling `csv_info` function. Output of `df.info()` is printed to obtain that information.

In [ ]:
def csv_info(url):
    df = pd.read_csv(url)
    print("\nSTATS:\n")
    df.info()

## Transfer data
pandas provides built-in functions that make it simple to read and extract data from a CSV file. `read_csv` receives the path of the file as parameter, it can be a URL or a local file. Columns can be specified if not all are required, with `usecols` paramter in `read_csv` method.

To avoid `Connection timed out`, data from CSV is read in chunks with `chunksize` parameter in `read_csv` method.

As the dataset has about eight thousand registries, data is divided into 8 chunks as `chunksize` is equal to `1000`.

Every chunk is saved as temporary CSV files and passed to `read_data` function that returns a list with data that is inserted into TerminusDB through `ìnsert_document` method.

In [ ]:
def insert_data(client, url):
    df = pd.read_csv(url, chunksize=1000, usecols = ['name', 'brewery_type', 'street', 'city', 'state', 'postal_code', 'website_url', 'phone', 'country', 'longitude', 'latitude'])
    for chunk in tqdm(df, desc='Transfering data'):
        csv = tempfile.NamedTemporaryFile()
        chunk.to_csv(csv)
        breweries = read_data(csv.name)
        client.insert_document(breweries,
                               commit_msg="Adding all breweries")

As some cells in the CSV don't have a value, when importing data it would be required to replace `NULL` values with '' if an integer or float is expected instead. `fillna` must be called to avoid `Not a number` errors.

For importing data, a for loop is used to iterate through the values in the CSV, create objects for each document in the schema, assign values to the corresponding variables, and append these values to the `breweries` list.

In [ ]:
    breweries = []
    df = pd.read_csv(csv)
    selection = df.fillna('')
    for index, row in selection.iterrows():
        # City
        city = City()
        city.name = row['city']
        breweries.append(city)

        # State
        state = State()
        state.name = row['state']
        breweries.append(state)

        # Country
        country = Country()
        country.name = row['country']
        breweries.append(country)
        
        # Coordinates
        coordinates = Coordinates()
        coordinates.latitude = 0 if row['latitude']=='' else row['latitude']
        coordinates.longitude = 0 if row['longitude']=='' else row['longitude']
        breweries.append(coordinates)

        # Address
        address = Address()
        address.street = row['street']
        address.city = city
        address.state = state
        address.country = country
        address.postal_code = row['postal_code']
        address.coordinates = [coordinates]
        breweries.append(address)

        # Brewery
        brewery = Brewery()
        brewery.name = row['name']
        brewery.type_of = Brewery_Type[row['brewery_type']]
        brewery.address_of = address
        brewery.phone = str(row['phone'])
        brewery.website_url = row['website_url']
        breweries.append(brewery)
    
    return breweries